In [1]:
!python -V

Python 3.9.19


In [23]:
import pandas as pd
import numpy as np

In [14]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    df['duration'] = (df['lpep_dropoff_datetime'] - df['lpep_pickup_datetime'])
    df['duration'] = df.duration.apply(lambda time_delta: time_delta.total_seconds()/60)
    df = df[(df['duration'] >= 1) & (df['duration'] <= 60)]
    
    categorical = ['PULocationID', 'DOLocationID']
    #numerical = ['trip_distance']
    df[categorical] = df[categorical].astype('str')
    
    return df

In [15]:
train_df = read_dataframe(filename = "./data/green_tripdata_2021-01.parquet")
valid_df = read_dataframe(filename = "./data/green_tripdata_2021-02.parquet")

In [16]:
train_df

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,duration
0,2,2021-01-01 00:15:56,2021-01-01 00:19:52,N,1.0,43,151,1.0,1.01,5.50,...,0.5,0.00,0.00,None,0.3,6.80,2.0,1.0,0.00,3.933333
1,2,2021-01-01 00:25:59,2021-01-01 00:34:44,N,1.0,166,239,1.0,2.53,10.00,...,0.5,2.81,0.00,None,0.3,16.86,1.0,1.0,2.75,8.750000
2,2,2021-01-01 00:45:57,2021-01-01 00:51:55,N,1.0,41,42,1.0,1.12,6.00,...,0.5,1.00,0.00,None,0.3,8.30,1.0,1.0,0.00,5.966667
3,2,2020-12-31 23:57:51,2021-01-01 00:04:56,N,1.0,168,75,1.0,1.99,8.00,...,0.5,0.00,0.00,None,0.3,9.30,2.0,1.0,0.00,7.083333
7,2,2021-01-01 00:26:31,2021-01-01 00:28:50,N,1.0,75,75,6.0,0.45,3.50,...,0.5,0.96,0.00,None,0.3,5.76,1.0,1.0,0.00,2.316667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76513,2,2021-01-31 21:38:00,2021-01-31 22:16:00,None,NaN,81,90,NaN,17.63,56.23,...,0.0,0.00,6.12,None,0.3,65.40,NaN,NaN,NaN,38.000000
76514,2,2021-01-31 22:43:00,2021-01-31 23:21:00,None,NaN,35,213,NaN,18.36,46.66,...,0.0,12.20,6.12,None,0.3,65.28,NaN,NaN,NaN,38.000000
76515,2,2021-01-31 22:16:00,2021-01-31 22:27:00,None,NaN,74,69,NaN,2.50,18.95,...,0.0,0.00,0.00,None,0.3,22.00,NaN,NaN,NaN,11.000000
76516,2,2021-01-31 23:10:00,2021-01-31 23:37:00,None,NaN,168,215,NaN,14.48,48.87,...,0.0,0.00,6.12,None,0.3,58.04,NaN,NaN,NaN,27.000000


In [17]:
train_df.columns

Index(['VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime',
       'store_and_fwd_flag', 'RatecodeID', 'PULocationID', 'DOLocationID',
       'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax',
       'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge',
       'total_amount', 'payment_type', 'trip_type', 'congestion_surcharge',
       'duration'],
      dtype='object')

In [18]:
train_df['PU_DO'] = train_df['PULocationID'] + "_" + train_df['DOLocationID']
valid_df['PU_DO'] = valid_df['PULocationID'] + "_" + valid_df['DOLocationID']

In [19]:
categorical = ['PU_DO', 'PULocationID', 'DOLocationID' ]
numerical = ['trip_distance']
target = 'duration'
train_df = train_df[categorical + numerical + [target]]
valid_df = valid_df[categorical + numerical + [target]]

In [20]:
from sklearn.preprocessing import LabelEncoder
le_dict = {}
for col_ in categorical:
    le = LabelEncoder()
    train_df[col_ + "_le"] = le.fit_transform(train_df[[col_]])
    le_dict[col_] = le

train_df.drop(columns = categorical, inplace=True)
train_df.dtypes

/home/ubuntu/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/tmp/ipykernel_3680/4210087121.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df[col_ + "_le"] = le.fit_transform(train_df[[col_]])
/home/ubuntu/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/tmp/ipykernel_

trip_distance      float64
duration           float64
PU_DO_le             int64
PULocationID_le      int64
DOLocationID_le      int64
dtype: object

In [25]:
for col_ in categorical:
    le = le_dict[col_]

    valid_df[col_] = np.where(valid_df[col_].isin(le.classes_), valid_df[col_], -99999)
    le.classes_=np.array([-99999] + le.classes_.tolist())
    
    valid_df[col_ + "_le"] = le.transform(valid_df[col_])

valid_df.drop(columns = categorical, inplace=True)

In [26]:
predictors = list(set(train_df.columns) - set(['duration']))
len(predictors)

4

In [67]:
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("duration_prediction_model_registry_example")

2024/05/30 03:46:36 INFO mlflow.tracking.fluent: Experiment with name 'duration_prediction_model_registry_example' does not exist. Creating a new experiment.


<Experiment: artifact_location='/home/ubuntu/code/mlops_zoomcamp_homework/mlops_zoomcamp/2-experiment-tracking/mlruns/2', creation_time=1717040796597, experiment_id='2', last_update_time=1717040796597, lifecycle_stage='active', name='duration_prediction_model_registry_example', tags={}>

# RANDOM FOREST REGRESSOR

In [68]:
with mlflow.start_run():
    from sklearn.ensemble import RandomForestRegressor
    
    params = {'n_estimators' : 100,
              'criterion' : 'squared_error',
              'max_depth' : 15,
              'max_features' : 1.0,
              'min_samples_leaf' : 10,
              #'min_samples_split' : 2,
              #'min_weight_fraction_leaf' : 0.0
             }

    mlflow.log_params(params)
    mlflow.set_tag('developer', 'felix')
    mlflow.set_tag('model', 'sklearn_random_forest')
    
    rf = RandomForestRegressor(**params)
    
    rf.fit(train_df[predictors], train_df[target] )
    
    val_pred = rf.predict(valid_df[predictors])
    
    from sklearn.metrics import mean_squared_error
    
    print(mean_squared_error(valid_df[target], val_pred, squared=False))
    mlflow.log_metric('val_rmse', mean_squared_error(valid_df[target], val_pred, squared=False))

6.430823271725588


/home/ubuntu/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/ubuntu/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [ ]:
with mlflow.start_run():
    from sklearn.ensemble import GradientBoostingRegressor
    
    params = {'loss': 'squared_error',
              'learning_rate': 0.1,
              'n_estimators': 100,
              'subsample': 0.7,
              'min_samples_split': 2,
              'min_samples_leaf': 1,
              'min_weight_fraction_leaf': 0.0,
              'max_depth': 3,
              'min_impurity_decrease': 0.0,
              'random_state': 1234,
              'max_features': 0.7,
              'alpha': 0.1,
              'verbose': 0
             }

    mlflow.log_params(params)
    mlflow.set_tag('developer', 'felix')
    mlflow.set_tag('model', 'sklearn_GBM')
    
    
    gbtr = GradientBoostingRegressor(**params)
    
    gbtr.fit(train_df[predictors], train_df[target] )
    
    val_pred = gbtr.predict(valid_df[predictors])
    
    from sklearn.metrics import mean_squared_error
    
    print(mean_squared_error(valid_df[target], val_pred, squared=False))
    mlflow.log_metric('val_rmse', mean_squared_error(valid_df[target], val_pred, squared=False))

In [65]:
from sklearn.ensemble import ExtraTreesRegressor

params = {'n_estimators' : 100,
          'criterion' : 'squared_error',
          'max_depth' : 15,
          'max_features' : 1.0,
          'min_samples_leaf' : 10,
          #'min_samples_split' : 2,
          #'min_weight_fraction_leaf' : 0.0
         }


xrf = ExtraTreesRegressor(**params)

xrf.fit(train_df[predictors], train_df[target] )

val_pred = xrf.predict(valid_df[predictors])

from sklearn.metrics import mean_squared_error

mean_squared_error(valid_df[target], val_pred, squared=False)

/home/ubuntu/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.238492208604264